In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

# Load tokenizer and model from Hugging Face
# `use_auth_token=True` ensures access to models that require authentication

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B", use_auth_token=True)

# Check if CUDA (GPU) is available, otherwise fallback to CPU

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Function to generate responses for a batch of JSON data
  Generates model responses for given input data.
    
  Args:
    json_data (list of dict): List of queries and associated context in JSON format.
    batch_size (int): Number of queries to process in each batch.

  Returns:
    list of dict: Responses including the query and the generated answer.

In [ ]:
def generate_responses(json_data, batch_size=5):
  responses = []
  tokenizer.pad_token = tokenizer.eos_token

  queries_with_context = [
    f"Question: {item['Query']}\nContext: {item['Context']}\nInstructions: {item.get('Additional Instructions', '')}\n"
    for item in json_data
  ]

  for i in range(0, len(queries_with_context), batch_size):
    batch_questions = queries_with_context[i:i + batch_size]
    inputs = tokenizer(batch_questions, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    outputs = model.generate(
      input_ids=inputs['input_ids'],
      attention_mask=inputs['attention_mask'],
      max_new_tokens=100,
      pad_token_id=tokenizer.eos_token_id,
      no_repeat_ngram_size=2
    )

    for output, item in zip(outputs, json_data[i:i + batch_size]):
      decoded_output = tokenizer.decode(output, skip_special_tokens=True)
      responses.append({
        "query": item['Query'],
        "response": decoded_output
      })

  return responses

# Load JSON file and process

In [ ]:
json_file = "./ValueTest.json"
df = pd.read_json(json_file)
json_data = df.iloc[:1].to_dict(orient='records')


In [ ]:
output_data = generate_responses(json_data)

# Save results

In [ ]:
with open('generated_responses.json', 'w') as f:
    pd.json.dump(output_data, f, indent=2)

In [ ]:
print("Responses saved to 'generated_responses.json'")

[
    {
        "id": 1871,
        "Query": "What is the US GAAP XBRL tag for Cost of Goods Sold as reported by Amazon.com Inc for the Fiscal Year ending in FY 2023? (Response format: XBRL tag, e.g., 'Answer: us-gaap:Depreciation')",
        "Context": "file:c-7.xml\n<{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities contextRef=\"c-7\" decimals=\"-6\" id=\"f-90\" unitRef=\"usd\">60157000000</{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities contextRef=\"c-8\" decimals=\"-6\" id=\"f-91\" unitRef=\"usd\">2565000000</{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities contextRef=\"c-1\" decimals=\"-6\" id=\"f-92\" unitRef=\"usd\">1488000000</{http://fasb.org/us-gaap/2023}PaymentsToAcquireMarketableSecurities>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities contextRef=\"c-7\" decimals=\"-6\" id=\"f-93\" unitRef=\"usd\">-58154000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities contextRef=\"c-8\" decimals=\"-6\" id=\"f-94\" unitRef=\"usd\">-37601000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities contextRef=\"c-1\" decimals=\"-6\" id=\"f-95\" unitRef=\"usd\">-49833000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInInvestingActivities>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock contextRef=\"c-7\" decimals=\"-6\" id=\"f-96\" unitRef=\"usd\">0</{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock contextRef=\"c-8\" decimals=\"-6\" id=\"f-97\" unitRef=\"usd\">6000000000</{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock contextRef=\"c-1\" decimals=\"-6\" id=\"f-98\" unitRef=\"usd\">0</{http://fasb.org/us-gaap/2023}PaymentsForRepurchaseOfCommonStock>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt contextRef=\"c-7\" decimals=\"-6\" id=\"f-99\" unitRef=\"usd\">7956000000</{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt contextRef=\"c-8\" decimals=\"-6\" id=\"f-100\" unitRef=\"usd\">41553000000</{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt contextRef=\"c-1\" decimals=\"-6\" id=\"f-101\" unitRef=\"usd\">18129000000</{http://fasb.org/us-gaap/2023}ProceedsFromShortTermDebt>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt contextRef=\"c-7\" decimals=\"-6\" id=\"f-102\" unitRef=\"usd\">7753000000</{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt contextRef=\"c-8\" decimals=\"-6\" id=\"f-103\" unitRef=\"usd\">37554000000</{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt contextRef=\"c-1\" decimals=\"-6\" id=\"f-104\" unitRef=\"usd\">25677000000</{http://fasb.org/us-gaap/2023}RepaymentsOfShortTermDebt>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt contextRef=\"c-7\" decimals=\"-6\" id=\"f-105\" unitRef=\"usd\">19003000000</{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt contextRef=\"c-8\" decimals=\"-6\" id=\"f-106\" unitRef=\"usd\">21166000000</{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt contextRef=\"c-1\" decimals=\"-6\" id=\"f-107\" unitRef=\"usd\">0</{http://fasb.org/us-gaap/2023}ProceedsFromIssuanceOfLongTermDebt>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt contextRef=\"c-7\" decimals=\"-6\" id=\"f-108\" unitRef=\"usd\">1590000000</{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt contextRef=\"c-8\" decimals=\"-6\" id=\"f-109\" unitRef=\"usd\">1258000000</{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt contextRef=\"c-1\" decimals=\"-6\" id=\"f-110\" unitRef=\"usd\">3676000000</{http://fasb.org/us-gaap/2023}RepaymentsOfLongTermDebt>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments contextRef=\"c-7\" decimals=\"-6\" id=\"f-111\" unitRef=\"usd\">11163000000</{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments contextRef=\"c-8\" decimals=\"-6\" id=\"f-112\" unitRef=\"usd\">7941000000</{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments contextRef=\"c-1\" decimals=\"-6\" id=\"f-113\" unitRef=\"usd\">4384000000</{http://fasb.org/us-gaap/2023}FinanceLeasePrincipalPayments>\nfile:c-7.xml\n<{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations contextRef=\"c-7\" decimals=\"-6\" id=\"f-114\" unitRef=\"usd\">162000000</{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations>\nfile:c-8.xml\n<{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations contextRef=\"c-8\" decimals=\"-6\" id=\"f-115\" unitRef=\"usd\">248000000</{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations>\nfile:c-1.xml\n<{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations contextRef=\"c-1\" decimals=\"-6\" id=\"f-116\" unitRef=\"usd\">271000000</{http://www.amazon.com/20231231}RepaymentsOfLongTermFinancingObligations>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities contextRef=\"c-7\" decimals=\"-6\" id=\"f-117\" unitRef=\"usd\">6291000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities contextRef=\"c-8\" decimals=\"-6\" id=\"f-118\" unitRef=\"usd\">9718000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities contextRef=\"c-1\" decimals=\"-6\" id=\"f-119\" unitRef=\"usd\">-15879000000</{http://fasb.org/us-gaap/2023}NetCashProvidedByUsedInFinancingActivities>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations contextRef=\"c-7\" decimals=\"-6\" id=\"f-120\" unitRef=\"usd\">-364000000</{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations contextRef=\"c-8\" decimals=\"-6\" id=\"f-121\" unitRef=\"usd\">-1093000000</{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations contextRef=\"c-1\" decimals=\"-6\" id=\"f-122\" unitRef=\"usd\">403000000</{http://fasb.org/us-gaap/2023}EffectOfExchangeRateOnCashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsIncludingDisposalGroupAndDiscontinuedOperations>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect contextRef=\"c-7\" decimals=\"-6\" id=\"f-123\" unitRef=\"usd\">-5900000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect contextRef=\"c-8\" decimals=\"-6\" id=\"f-124\" unitRef=\"usd\">17776000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect contextRef=\"c-1\" decimals=\"-6\" id=\"f-125\" unitRef=\"usd\">19637000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect>\nfile:c-5.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents contextRef=\"c-5\" decimals=\"-6\" id=\"f-126\" unitRef=\"usd\">36477000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents>\nfile:c-6.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents contextRef=\"c-6\" decimals=\"-6\" id=\"f-127\" unitRef=\"usd\">54253000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents>\nfile:c-9.xml\n<{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents contextRef=\"c-9\" decimals=\"-6\" id=\"f-128\" unitRef=\"usd\">73890000000</{http://fasb.org/us-gaap/2023}CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalents>\nfile:c-10.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-10\" decimals=\"-6\" id=\"f-129\" unitRef=\"usd\">241787000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-11.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-11\" decimals=\"-6\" id=\"f-130\" unitRef=\"usd\">242901000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-12.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-12\" decimals=\"-6\" id=\"f-131\" unitRef=\"usd\">255887000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-13\" decimals=\"-6\" id=\"f-132\" unitRef=\"usd\">228035000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-14\" decimals=\"-6\" id=\"f-133\" unitRef=\"usd\">271082000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-15.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-15\" decimals=\"-6\" id=\"f-134\" unitRef=\"usd\">318898000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-7\" decimals=\"-6\" id=\"f-135\" unitRef=\"usd\">469822000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-8\" decimals=\"-6\" id=\"f-136\" unitRef=\"usd\">513983000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-1\" decimals=\"-6\" id=\"f-137\" unitRef=\"usd\">574785000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-7\" decimals=\"-6\" id=\"f-138\" unitRef=\"usd\">272344000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-8\" decimals=\"-6\" id=\"f-139\" unitRef=\"usd\">288831000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-1\" decimals=\"-6\" id=\"f-140\" unitRef=\"usd\">304739000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-7.xml\n<{http://www.amazon.com/20231231}FulfillmentExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-141\" unitRef=\"usd\">75111000000</{http://www.amazon.com/20231231}FulfillmentExpense>\nfile:c-8.xml\n<{http://www.amazon.com/20231231}FulfillmentExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-142\" unitRef=\"usd\">84299000000</{http://www.amazon.com/20231231}FulfillmentExpense>\nfile:c-1.xml\n<{http://www.amazon.com/20231231}FulfillmentExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-143\" unitRef=\"usd\">90619000000</{http://www.amazon.com/20231231}FulfillmentExpense>\nfile:c-7.xml\n<{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-144\" unitRef=\"usd\">56052000000</{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense>\nfile:c-8.xml\n<{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-145\" unitRef=\"usd\">73213000000</{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense>\nfile:c-1.xml\n<{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-146\" unitRef=\"usd\">85622000000</{http://www.amazon.com/20231231}TechnologyAndInfrastructureExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}MarketingExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-147\" unitRef=\"usd\">32551000000</{http://fasb.org/us-gaap/2023}MarketingExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}MarketingExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-148\" unitRef=\"usd\">42238000000</{http://fasb.org/us-gaap/2023}MarketingExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}MarketingExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-149\" unitRef=\"usd\">44370000000</{http://fasb.org/us-gaap/2023}MarketingExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-150\" unitRef=\"usd\">8823000000</{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-151\" unitRef=\"usd\">11891000000</{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-152\" unitRef=\"usd\">11816000000</{http://fasb.org/us-gaap/2023}GeneralAndAdministrativeExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet contextRef=\"c-7\" decimals=\"-6\" id=\"f-153\" unitRef=\"usd\">-62000000</{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet contextRef=\"c-8\" decimals=\"-6\" id=\"f-154\" unitRef=\"usd\">-1263000000</{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet contextRef=\"c-1\" decimals=\"-6\" id=\"f-155\" unitRef=\"usd\">-767000000</{http://fasb.org/us-gaap/2023}OtherOperatingIncomeExpenseNet>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-7\" decimals=\"-6\" id=\"f-156\" unitRef=\"usd\">444943000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-8\" decimals=\"-6\" id=\"f-157\" unitRef=\"usd\">501735000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-1\" decimals=\"-6\" id=\"f-158\" unitRef=\"usd\">537933000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-7\" decimals=\"-6\" id=\"f-159\" unitRef=\"usd\">24879000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-8\" decimals=\"-6\" id=\"f-160\" unitRef=\"usd\">12248000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-1\" decimals=\"-6\" id=\"f-161\" unitRef=\"usd\">36852000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest contextRef=\"c-7\" decimals=\"-6\" id=\"f-162\" unitRef=\"usd\">448000000</{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest contextRef=\"c-8\" decimals=\"-6\" id=\"f-163\" unitRef=\"usd\">989000000</{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest contextRef=\"c-1\" decimals=\"-6\" id=\"f-164\" unitRef=\"usd\">2949000000</{http://fasb.org/us-gaap/2023}InvestmentIncomeInterest>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}InterestExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-165\" unitRef=\"usd\">1809000000</{http://fasb.org/us-gaap/2023}InterestExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}InterestExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-166\" unitRef=\"usd\">2367000000</{http://fasb.org/us-gaap/2023}InterestExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}InterestExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-167\" unitRef=\"usd\">3182000000</{http://fasb.org/us-gaap/2023}InterestExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-168\" unitRef=\"usd\">14633000000</{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-169\" unitRef=\"usd\">-16806000000</{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-170\" unitRef=\"usd\">938000000</{http://fasb.org/us-gaap/2023}OtherNonoperatingIncomeExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-171\" unitRef=\"usd\">13272000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-7\" decimals=\"-6\" id=\"f-172\" unitRef=\"usd\">13272000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-173\" unitRef=\"usd\">-18184000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-8\" decimals=\"-6\" id=\"f-174\" unitRef=\"usd\">-18184000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-175\" unitRef=\"usd\">705000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-7\" decimals=\"-6\" id=\"f-176\" unitRef=\"usd\">38151000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-8\" decimals=\"-6\" id=\"f-177\" unitRef=\"usd\">-5936000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-1\" decimals=\"-6\" id=\"f-178\" unitRef=\"usd\">37557000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-7\" decimals=\"-6\" id=\"f-179\" unitRef=\"usd\">4791000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-8\" decimals=\"-6\" id=\"f-180\" unitRef=\"usd\">-3217000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-1\" decimals=\"-6\" id=\"f-181\" unitRef=\"usd\">7120000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments contextRef=\"c-7\" decimals=\"-6\" id=\"f-182\" unitRef=\"usd\">4000000</{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments contextRef=\"c-8\" decimals=\"-6\" id=\"f-183\" unitRef=\"usd\">-3000000</{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments contextRef=\"c-1\" decimals=\"-6\" id=\"f-184\" unitRef=\"usd\">-12000000</{http://fasb.org/us-gaap/2023}IncomeLossFromEquityMethodInvestments>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-7\" decimals=\"-6\" id=\"f-185\" unitRef=\"usd\">33364000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-8\" decimals=\"-6\" id=\"f-186\" unitRef=\"usd\">-2722000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-1\" decimals=\"-6\" id=\"f-187\" unitRef=\"usd\">30425000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-7\" decimals=\"2\" id=\"f-188\" unitRef=\"usdPerShare\">3.30</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-8\" decimals=\"2\" id=\"f-189\" unitRef=\"usdPerShare\">-0.27</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-1\" decimals=\"2\" id=\"f-190\" unitRef=\"usdPerShare\">2.95</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\n",
        "Additional Instructions": "The answer must be the exact consolidated tag found in the source document.             Also take any company-created or customized tags into consideration if they are relevant.             Provide the final XBRL tag as it appears in the source.",
        "Response Formats": "Answer: {XBRL tag}"
    },
    {
        "id": 2966,
        "Query": "What is the US GAAP XBRL tag for Cost of Goods Sold as reported by Amgen Inc for the Fiscal Year ending in FY 2023? (Response format: XBRL tag, e.g., 'Answer: us-gaap:Depreciation')",
        "Context": "file:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentPeriodEndDate contextRef=\"c-1\" decimals=\"None\" id=\"f-3\" unitRef=\"None\">2023-12-31</{http://xbrl.sec.gov/dei/2023}DocumentPeriodEndDate>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}CurrentFiscalYearEndDate contextRef=\"c-1\" decimals=\"None\" id=\"f-4\" unitRef=\"None\">--12-31</{http://xbrl.sec.gov/dei/2023}CurrentFiscalYearEndDate>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentTransitionReport contextRef=\"c-1\" decimals=\"None\" id=\"f-5\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}DocumentTransitionReport>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityFileNumber contextRef=\"c-1\" decimals=\"None\" id=\"f-6\" unitRef=\"None\">001-37702</{http://xbrl.sec.gov/dei/2023}EntityFileNumber>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityRegistrantName contextRef=\"c-1\" decimals=\"None\" id=\"f-7\" unitRef=\"None\">Amgen Inc.</{http://xbrl.sec.gov/dei/2023}EntityRegistrantName>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityIncorporationStateCountryCode contextRef=\"c-1\" decimals=\"None\" id=\"f-8\" unitRef=\"None\">DE</{http://xbrl.sec.gov/dei/2023}EntityIncorporationStateCountryCode>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityTaxIdentificationNumber contextRef=\"c-1\" decimals=\"None\" id=\"f-9\" unitRef=\"None\">95-3540776</{http://xbrl.sec.gov/dei/2023}EntityTaxIdentificationNumber>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityAddressAddressLine1 contextRef=\"c-1\" decimals=\"None\" id=\"f-10\" unitRef=\"None\">One Amgen Center Drive</{http://xbrl.sec.gov/dei/2023}EntityAddressAddressLine1>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityAddressPostalZipCode contextRef=\"c-1\" decimals=\"None\" id=\"f-11\" unitRef=\"None\">91320-1799</{http://xbrl.sec.gov/dei/2023}EntityAddressPostalZipCode>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityAddressCityOrTown contextRef=\"c-1\" decimals=\"None\" id=\"f-12\" unitRef=\"None\">Thousand Oaks</{http://xbrl.sec.gov/dei/2023}EntityAddressCityOrTown>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityAddressStateOrProvince contextRef=\"c-1\" decimals=\"None\" id=\"f-13\" unitRef=\"None\">CA</{http://xbrl.sec.gov/dei/2023}EntityAddressStateOrProvince>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}CityAreaCode contextRef=\"c-1\" decimals=\"None\" id=\"f-14\" unitRef=\"None\">805</{http://xbrl.sec.gov/dei/2023}CityAreaCode>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}LocalPhoneNumber contextRef=\"c-1\" decimals=\"None\" id=\"f-15\" unitRef=\"None\">447-1000</{http://xbrl.sec.gov/dei/2023}LocalPhoneNumber>\nfile:c-2.xml\n<{http://xbrl.sec.gov/dei/2023}Security12bTitle contextRef=\"c-2\" decimals=\"None\" id=\"f-16\" unitRef=\"None\">Common stock, $0.0001 par value</{http://xbrl.sec.gov/dei/2023}Security12bTitle>\nfile:c-2.xml\n<{http://xbrl.sec.gov/dei/2023}TradingSymbol contextRef=\"c-2\" decimals=\"None\" id=\"f-17\" unitRef=\"None\">AMGN</{http://xbrl.sec.gov/dei/2023}TradingSymbol>\nfile:c-2.xml\n<{http://xbrl.sec.gov/dei/2023}SecurityExchangeName contextRef=\"c-2\" decimals=\"None\" id=\"f-18\" unitRef=\"None\">NASDAQ</{http://xbrl.sec.gov/dei/2023}SecurityExchangeName>\nfile:c-3.xml\n<{http://xbrl.sec.gov/dei/2023}Security12bTitle contextRef=\"c-3\" decimals=\"None\" id=\"f-19\" unitRef=\"None\">2.00% Senior Notes due 2026</{http://xbrl.sec.gov/dei/2023}Security12bTitle>\nfile:c-3.xml\n<{http://xbrl.sec.gov/dei/2023}TradingSymbol contextRef=\"c-3\" decimals=\"None\" id=\"f-20\" unitRef=\"None\">AMGN26</{http://xbrl.sec.gov/dei/2023}TradingSymbol>\nfile:c-3.xml\n<{http://xbrl.sec.gov/dei/2023}SecurityExchangeName contextRef=\"c-3\" decimals=\"None\" id=\"f-21\" unitRef=\"None\">NASDAQ</{http://xbrl.sec.gov/dei/2023}SecurityExchangeName>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityWellKnownSeasonedIssuer contextRef=\"c-1\" decimals=\"None\" id=\"f-22\" unitRef=\"None\">Yes</{http://xbrl.sec.gov/dei/2023}EntityWellKnownSeasonedIssuer>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityVoluntaryFilers contextRef=\"c-1\" decimals=\"None\" id=\"f-23\" unitRef=\"None\">No</{http://xbrl.sec.gov/dei/2023}EntityVoluntaryFilers>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityCurrentReportingStatus contextRef=\"c-1\" decimals=\"None\" id=\"f-24\" unitRef=\"None\">Yes</{http://xbrl.sec.gov/dei/2023}EntityCurrentReportingStatus>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityInteractiveDataCurrent contextRef=\"c-1\" decimals=\"None\" id=\"f-25\" unitRef=\"None\">Yes</{http://xbrl.sec.gov/dei/2023}EntityInteractiveDataCurrent>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityFilerCategory contextRef=\"c-1\" decimals=\"None\" id=\"f-26\" unitRef=\"None\">Large Accelerated Filer</{http://xbrl.sec.gov/dei/2023}EntityFilerCategory>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntitySmallBusiness contextRef=\"c-1\" decimals=\"None\" id=\"f-27\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}EntitySmallBusiness>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityEmergingGrowthCompany contextRef=\"c-1\" decimals=\"None\" id=\"f-28\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}EntityEmergingGrowthCompany>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}IcfrAuditorAttestationFlag contextRef=\"c-1\" decimals=\"None\" id=\"f-29\" unitRef=\"None\">true</{http://xbrl.sec.gov/dei/2023}IcfrAuditorAttestationFlag>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentFinStmtErrorCorrectionFlag contextRef=\"c-1\" decimals=\"None\" id=\"f-30\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}DocumentFinStmtErrorCorrectionFlag>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityShellCompany contextRef=\"c-1\" decimals=\"None\" id=\"f-31\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}EntityShellCompany>\nfile:c-4.xml\n<{http://xbrl.sec.gov/dei/2023}EntityPublicFloat contextRef=\"c-4\" decimals=\"0\" id=\"f-32\" unitRef=\"usd\">118556278405</{http://xbrl.sec.gov/dei/2023}EntityPublicFloat>\nfile:c-5.xml\n<{http://xbrl.sec.gov/dei/2023}EntityCommonStockSharesOutstanding contextRef=\"c-5\" decimals=\"0\" id=\"f-33\" unitRef=\"shares\">535918901</{http://xbrl.sec.gov/dei/2023}EntityCommonStockSharesOutstanding>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentsIncorporatedByReferenceTextBlock contextRef=\"c-1\" decimals=\"None\" id=\"f-34\" unitRef=\"None\"><div style=\"margin-top:2pt;text-align:center\"><span style=\"color:#000000;font-family:'Times New Roman',sans-serif;font-size:10pt;font-weight:700;line-height:120%\">DOCUMENTS INCORPORATED BY REFERENCE</span></div><div style=\"margin-top:2pt;text-align:justify;text-indent:24.75pt\"><span style=\"color:#000000;font-family:'Times New Roman',sans-serif;font-size:10pt;font-weight:400;line-height:120%\">Specified portions of the registrant\u2019s Proxy Statement with respect to the 2024 Annual Meeting of Stockholders to be held on</span><span style=\"color:#ee2724;font-family:'Times New Roman',sans-serif;font-size:10pt;font-weight:400;line-height:120%\"> </span><span style=\"color:#000000;font-family:'Times New Roman',sans-serif;font-size:10pt;font-weight:400;line-height:120%\">May 31, 2024, are incorporated by reference into Part\u00a0III of this annual report.</span></div></{http://xbrl.sec.gov/dei/2023}DocumentsIncorporatedByReferenceTextBlock>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}AuditorName contextRef=\"c-1\" decimals=\"None\" id=\"f-39\" unitRef=\"None\">Ernst & Young LLP</{http://xbrl.sec.gov/dei/2023}AuditorName>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}AuditorLocation contextRef=\"c-1\" decimals=\"None\" id=\"f-40\" unitRef=\"None\">Los Angeles, California</{http://xbrl.sec.gov/dei/2023}AuditorLocation>\nfile:c-6.xml\n<{http://xbrl.sec.gov/ecd/2023}NonRule10b51ArrAdoptedFlag contextRef=\"c-6\" decimals=\"None\" id=\"f-41\" unitRef=\"None\">false</{http://xbrl.sec.gov/ecd/2023}NonRule10b51ArrAdoptedFlag>\nfile:c-6.xml\n<{http://xbrl.sec.gov/ecd/2023}Rule10b51ArrTrmntdFlag contextRef=\"c-6\" decimals=\"None\" id=\"f-42\" unitRef=\"None\">false</{http://xbrl.sec.gov/ecd/2023}Rule10b51ArrTrmntdFlag>\nfile:c-6.xml\n<{http://xbrl.sec.gov/ecd/2023}NonRule10b51ArrTrmntdFlag contextRef=\"c-6\" decimals=\"None\" id=\"f-43\" unitRef=\"None\">false</{http://xbrl.sec.gov/ecd/2023}NonRule10b51ArrTrmntdFlag>\nfile:c-6.xml\n<{http://xbrl.sec.gov/ecd/2023}Rule10b51ArrAdoptedFlag contextRef=\"c-6\" decimals=\"None\" id=\"f-44\" unitRef=\"None\">false</{http://xbrl.sec.gov/ecd/2023}Rule10b51ArrAdoptedFlag>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}AuditorFirmId contextRef=\"c-1\" decimals=\"None\" id=\"f-45\" unitRef=\"None\">42</{http://xbrl.sec.gov/dei/2023}AuditorFirmId>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}AuditorName contextRef=\"c-1\" decimals=\"None\" id=\"f-46\" unitRef=\"None\">Ernst & Young LLP</{http://xbrl.sec.gov/dei/2023}AuditorName>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}AuditorLocation contextRef=\"c-1\" decimals=\"None\" id=\"f-47\" unitRef=\"None\">Los Angeles, California</{http://xbrl.sec.gov/dei/2023}AuditorLocation>\nfile:c-7.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-7\" decimals=\"-6\" id=\"f-48\" unitRef=\"usd\">26910000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-8.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-8\" decimals=\"-6\" id=\"f-49\" unitRef=\"usd\">24801000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-9.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-9\" decimals=\"-6\" id=\"f-50\" unitRef=\"usd\">24297000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-10.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-10\" decimals=\"-6\" id=\"f-51\" unitRef=\"usd\">1280000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-11.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-11\" decimals=\"-6\" id=\"f-52\" unitRef=\"usd\">1522000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-12.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-12\" decimals=\"-6\" id=\"f-53\" unitRef=\"usd\">1682000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-1\" decimals=\"-6\" id=\"f-54\" unitRef=\"usd\">28190000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-13\" decimals=\"-6\" id=\"f-55\" unitRef=\"usd\">26323000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-14\" decimals=\"-6\" id=\"f-56\" unitRef=\"usd\">25979000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-1\" decimals=\"-6\" id=\"f-57\" unitRef=\"usd\">8451000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-13\" decimals=\"-6\" id=\"f-58\" unitRef=\"usd\">6406000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold contextRef=\"c-14\" decimals=\"-6\" id=\"f-59\" unitRef=\"usd\">6454000000</{http://fasb.org/us-gaap/2023}CostOfGoodsAndServicesSold>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost contextRef=\"c-1\" decimals=\"-6\" id=\"f-60\" unitRef=\"usd\">4784000000</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost contextRef=\"c-13\" decimals=\"-6\" id=\"f-61\" unitRef=\"usd\">4434000000</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost contextRef=\"c-14\" decimals=\"-6\" id=\"f-62\" unitRef=\"usd\">4819000000</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentExpenseExcludingAcquiredInProcessCost>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff contextRef=\"c-1\" decimals=\"-6\" id=\"f-63\" unitRef=\"usd\">0</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff contextRef=\"c-13\" decimals=\"-6\" id=\"f-64\" unitRef=\"usd\">0</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff contextRef=\"c-14\" decimals=\"-6\" id=\"f-65\" unitRef=\"usd\">1505000000</{http://fasb.org/us-gaap/2023}ResearchAndDevelopmentAssetAcquiredOtherThanThroughBusinessCombinationWrittenOff>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-66\" unitRef=\"usd\">6179000000</{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense contextRef=\"c-13\" decimals=\"-6\" id=\"f-67\" unitRef=\"usd\">5414000000</{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense contextRef=\"c-14\" decimals=\"-6\" id=\"f-68\" unitRef=\"usd\">5368000000</{http://fasb.org/us-gaap/2023}SellingGeneralAndAdministrativeExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}OtherGeneralExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-69\" unitRef=\"usd\">879000000</{http://fasb.org/us-gaap/2023}OtherGeneralExpense>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}OtherGeneralExpense contextRef=\"c-13\" decimals=\"-6\" id=\"f-70\" unitRef=\"usd\">503000000</{http://fasb.org/us-gaap/2023}OtherGeneralExpense>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}OtherGeneralExpense contextRef=\"c-14\" decimals=\"-6\" id=\"f-71\" unitRef=\"usd\">194000000</{http://fasb.org/us-gaap/2023}OtherGeneralExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-1\" decimals=\"-6\" id=\"f-72\" unitRef=\"usd\">20293000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-13\" decimals=\"-6\" id=\"f-73\" unitRef=\"usd\">16757000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}CostsAndExpenses contextRef=\"c-14\" decimals=\"-6\" id=\"f-74\" unitRef=\"usd\">18340000000</{http://fasb.org/us-gaap/2023}CostsAndExpenses>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-1\" decimals=\"-6\" id=\"f-75\" unitRef=\"usd\">7897000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-13\" decimals=\"-6\" id=\"f-76\" unitRef=\"usd\">9566000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}OperatingIncomeLoss contextRef=\"c-14\" decimals=\"-6\" id=\"f-77\" unitRef=\"usd\">7639000000</{http://fasb.org/us-gaap/2023}OperatingIncomeLoss>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}InterestExpenseDebt contextRef=\"c-1\" decimals=\"-6\" id=\"f-78\" unitRef=\"usd\">2875000000</{http://fasb.org/us-gaap/2023}InterestExpenseDebt>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}InterestExpenseDebt contextRef=\"c-13\" decimals=\"-6\" id=\"f-79\" unitRef=\"usd\">1406000000</{http://fasb.org/us-gaap/2023}InterestExpenseDebt>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}InterestExpenseDebt contextRef=\"c-14\" decimals=\"-6\" id=\"f-80\" unitRef=\"usd\">1197000000</{http://fasb.org/us-gaap/2023}InterestExpenseDebt>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-1\" decimals=\"-6\" id=\"f-81\" unitRef=\"usd\">2833000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-13\" decimals=\"-6\" id=\"f-82\" unitRef=\"usd\">-814000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense contextRef=\"c-14\" decimals=\"-6\" id=\"f-83\" unitRef=\"usd\">259000000</{http://fasb.org/us-gaap/2023}NonoperatingIncomeExpense>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-1\" decimals=\"-6\" id=\"f-84\" unitRef=\"usd\">7855000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-13\" decimals=\"-6\" id=\"f-85\" unitRef=\"usd\">7346000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments contextRef=\"c-14\" decimals=\"-6\" id=\"f-86\" unitRef=\"usd\">6701000000</{http://fasb.org/us-gaap/2023}IncomeLossFromContinuingOperationsBeforeIncomeTaxesMinorityInterestAndIncomeLossFromEquityMethodInvestments>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-1\" decimals=\"-6\" id=\"f-87\" unitRef=\"usd\">1138000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-13\" decimals=\"-6\" id=\"f-88\" unitRef=\"usd\">794000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit contextRef=\"c-14\" decimals=\"-6\" id=\"f-89\" unitRef=\"usd\">808000000</{http://fasb.org/us-gaap/2023}IncomeTaxExpenseBenefit>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-1\" decimals=\"-6\" id=\"f-90\" unitRef=\"usd\">6717000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-13\" decimals=\"-6\" id=\"f-91\" unitRef=\"usd\">6552000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-14\" decimals=\"-6\" id=\"f-92\" unitRef=\"usd\">5893000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-1\" decimals=\"2\" id=\"f-93\" unitRef=\"usdPerShare\">12.56</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-13\" decimals=\"2\" id=\"f-94\" unitRef=\"usdPerShare\">12.18</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareBasic contextRef=\"c-14\" decimals=\"2\" id=\"f-95\" unitRef=\"usdPerShare\">10.34</{http://fasb.org/us-gaap/2023}EarningsPerShareBasic>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted contextRef=\"c-1\" decimals=\"2\" id=\"f-96\" unitRef=\"usdPerShare\">12.49</{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted contextRef=\"c-13\" decimals=\"2\" id=\"f-97\" unitRef=\"usdPerShare\">12.11</{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted contextRef=\"c-14\" decimals=\"2\" id=\"f-98\" unitRef=\"usdPerShare\">10.28</{http://fasb.org/us-gaap/2023}EarningsPerShareDiluted>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic contextRef=\"c-1\" decimals=\"-6\" id=\"f-99\" unitRef=\"shares\">535000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic contextRef=\"c-13\" decimals=\"-6\" id=\"f-100\" unitRef=\"shares\">538000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic contextRef=\"c-14\" decimals=\"-6\" id=\"f-101\" unitRef=\"shares\">570000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfSharesOutstandingBasic>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding contextRef=\"c-1\" decimals=\"-6\" id=\"f-102\" unitRef=\"shares\">538000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding contextRef=\"c-13\" decimals=\"-6\" id=\"f-103\" unitRef=\"shares\">541000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding contextRef=\"c-14\" decimals=\"-6\" id=\"f-104\" unitRef=\"shares\">573000000</{http://fasb.org/us-gaap/2023}WeightedAverageNumberOfDilutedSharesOutstanding>\nfile:c-1.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-1\" decimals=\"-6\" id=\"f-105\" unitRef=\"usd\">6717000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-13.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-13\" decimals=\"-6\" id=\"f-106\" unitRef=\"usd\">6552000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}NetIncomeLoss contextRef=\"c-14\" decimals=\"-6\" id=\"f-107\" unitRef=\"usd\">5893000000</{http://fasb.org/us-gaap/2023}NetIncomeLoss>\n",
        "Additional Instructions": "The answer must be the exact consolidated tag found in the source document.             Also take any company-created or customized tags into consideration if they are relevant.             Provide the final XBRL tag as it appears in the source.",
        "Response Formats": "Answer: {XBRL tag}"
    },
    {
        "id": 1378,
        "Query": "What is the formula for the Gross Profit Margin of Apple Inc, formatted with the relevant US GAAP XBRL tags, for the fiscal year ending in FY 2023? (Response format: Formula with XBRL tags, e.g., 'Answer: (us-gaap:NetIncome / us-gaap:TotalRevenue) * 100')",
        "Context": "file:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}IcfrAuditorAttestationFlag contextRef=\"c-1\" decimals=\"None\" id=\"f-53\" unitRef=\"None\">true</{http://xbrl.sec.gov/dei/2023}IcfrAuditorAttestationFlag>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentFinStmtErrorCorrectionFlag contextRef=\"c-1\" decimals=\"None\" id=\"f-54\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}DocumentFinStmtErrorCorrectionFlag>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}EntityShellCompany contextRef=\"c-1\" decimals=\"None\" id=\"f-55\" unitRef=\"None\">false</{http://xbrl.sec.gov/dei/2023}EntityShellCompany>\nfile:c-12.xml\n<{http://xbrl.sec.gov/dei/2023}EntityPublicFloat contextRef=\"c-12\" decimals=\"-6\" id=\"f-56\" unitRef=\"usd\">2591165000000</{http://xbrl.sec.gov/dei/2023}EntityPublicFloat>\nfile:c-13.xml\n<{http://xbrl.sec.gov/dei/2023}EntityCommonStockSharesOutstanding contextRef=\"c-13\" decimals=\"-3\" id=\"f-57\" unitRef=\"shares\">15552752000</{http://xbrl.sec.gov/dei/2023}EntityCommonStockSharesOutstanding>\nfile:c-1.xml\n<{http://xbrl.sec.gov/dei/2023}DocumentsIncorporatedByReferenceTextBlock contextRef=\"c-1\" decimals=\"None\" id=\"f-58\" unitRef=\"None\">Portions of the Registrant\u2019s definitive proxy statement relating to its 2024 annual meeting of shareholders are incorporated by reference into Part III of this Annual Report on Form 10-K where indicated. The Registrant\u2019s definitive proxy statement will be filed with the U.S. Securities and Exchange Commission within 120 days after the end of the fiscal year to which this report relates.</{http://xbrl.sec.gov/dei/2023}DocumentsIncorporatedByReferenceTextBlock>\nfile:c-14.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-14\" decimals=\"-6\" id=\"f-63\" unitRef=\"usd\">298085000000</{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax>\nfile:c-15.xml\n<{http://fasb.org/us-gaap/2023}RevenueFromContractWithCustomerExcludingAssessedTax contextRef=\"c-15\" decimals=\"-6\" id=\"f-64\" unitRef=\"usd\">316199000000</{http://fasb.org/us-gaap/2023}RevenueFromContract... 
    }
}